# Mass ET Estimates from CCDC

### Reads in a folder of CCDC rasters from GEE and writes folder of ET estimates to Google Drive
#### Input rasters must be 4 bands in order of brightness temperature, NDVI, NDWI and water mask

In [1]:
import numpy as np
import rasterio
from rasterio.plot import show
import xarray as xr
import pandas as pd

import pprint

from IPython.display import Image
import openet.ssebop as ssebop

import ee
import gcloud
# Initialize the library.
#ee.Authenticate()
#ee.Authenticate(auth_mode='gcloud')
ee.Initialize()

In [2]:
#Image perameters and color scale
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
viridis_palette = ['440154', '433982', '30678D', '218F8B', '36B677', '8ED542', 'FDE725']

image_size = 768

# Define Functions for getting T surface

### Needed to convert Surface Brightness Temperatures to Land Surface Temperatures

In [3]:
def CCDC_emissivity(ccdcndvi):
    """Emissivity as a function of NDVI

    Parameters
    ----------
    CCDC Synthetic NDVI : ee.Image
        "Prepped" Landsat image with standardized band names.

    Returns
    -------
    ee.Image

    References
    ----------
    .. [Sobrino2004] Sobrino, J., J. Jiménez-Muñoz, & L. Paolini (2004).
        Land surface temperature retrieval from LANDSAT TM 5.
        Remote Sensing of Environment, 90(4), 434-440.
        https://doi.org/10.1016/j.rse.2004.02.003

    """
    ndvi_img = ccdcndvi
    Pv = ndvi_img.expression('((ndvi - 0.2) / 0.3) ** 2', {'ndvi': ndvi_img})
    # ndviRangevalue = ndvi_image.where(
    #     ndvi_image.gte(0.2).And(ndvi_image.lte(0.5)), ndvi_image)
    # Pv = ndviRangevalue.expression(
    #     '(((ndviRangevalue - 0.2) / 0.3) ** 2',
    #     {'ndviRangevalue':ndviRangevalue})

    # Assuming typical Soil Emissivity of 0.97 and Veg Emissivity of 0.99
    #   and shape Factor mean value of 0.553
    dE = Pv.expression(
        '(1 - 0.97) * (1 - Pv) * (0.55 * 0.99)', {'Pv': Pv})
    RangeEmiss = dE.expression(
        '(0.99 * Pv) + (0.97 * (1 - Pv)) + dE', {'Pv': Pv, 'dE': dE})

    # RangeEmiss = 0.989 # dE.expression(
    #  '((0.99 * Pv) + (0.97 * (1 - Pv)) + dE)', {'Pv':Pv, 'dE':dE})
    return ndvi_img\
        .where(ndvi_img.lt(0), 0.985)\
        .where(ndvi_img.gte(0).And(ndvi_img.lt(0.2)), 0.977)\
        .where(ndvi_img.gt(0.5), 0.99)\
        .where(ndvi_img.gte(0.2).And(ndvi_img.lte(0.5)), RangeEmiss)\
        .clamp(0.977, 0.99)\
        .rename(['emissivity'])



def CCDC_lst(brightnessc, ndvic):
    """Emissivity corrected land surface temperature (LST) from brightness Ts.

    Parameters
    ----------
    landsat_image : ee.Image
        "Prepped" Landsat image with standardized band names.
        Image must also have 'k1_constant' and 'k2_constant' properties.

    Returns
    -------
    ee.Image

    Notes
    -----
    The corrected radiation coefficients were derived from a small number
    of scenes in southern Idaho [Allen2007] and may not be appropriate for
    other areas.

    References
    ----------
    .. [Allen2007] R. Allen, M. Tasumi, R. Trezza (2007),
        Satellite-Based Energy Balance for Mapping Evapotranspiration with
        Internalized Calibration (METRIC) Model,
        Journal of Irrigation and Drainage Engineering, Vol 133(4),
        http://dx.doi.org/10.1061/(ASCE)0733-9437(2007)133:4(380)

    Notes
    -----
    tnb = 0.866   # narrow band transmissivity of air
    rp = 0.91     # path radiance
    rsky = 1.32   # narrow band clear sky downward thermal radiation

    """
    # Get properties from image
    k1 = ee.Number(666.09)   #K1 and K2 at https://doi.org/10.3390/cli3030563
    k2 = ee.Number(1282.71)  #K1 and K2 at https://doi.org/10.3390/cli3030563 
    
    #k1 = ee.Number(ee.Image(landsat_image).get('k1_constant')) #Talk to Heather about constants
    #k2 = ee.Number(ee.Image(landsat_image).get('k2_constant')) #Find right value for constant 

    ts_brightness = brightnessc
    emissivity_img = CCDC_emissivity(ndvic)

    # First back out radiance from brightness temperature
    # Then recalculate emissivity corrected Ts
    thermal_rad_toa = ts_brightness.expression(
        'k1 / (exp(k2 / ts_brightness) - 1)',
        {'ts_brightness': ts_brightness, 'k1': k1, 'k2': k2})

    rc = thermal_rad_toa.expression(
        '((thermal_rad_toa - rp) / tnb) - ((1 - emiss) * rsky)',
        {
            'thermal_rad_toa': thermal_rad_toa,
            'emiss': emissivity_img,
            'rp': 0.91, 'tnb': 0.866, 'rsky': 1.32,
        })
    lst = rc.expression(
        'k2 / log(emiss * k1 / rc + 1)',
        {'emiss': emissivity_img, 'rc': rc, 'k1': k1, 'k2': k2})

    return lst.rename(['lst'])

def get_water_mask(landsat_image):
    """
    Extract water mask from the Landsat Collection 2 SR QA_PIXEL band.
    :return: ee.Image
    """

    img = ee.Image(landsat_image)
    qa_img = img.select(['QA_PIXEL'])
    water_mask = qa_img.rightShift(7).bitwiseAnd(1).neq(0)
    return water_mask.rename(['qa_water'])

# Develop ETa estimates

In [4]:
#Get List of files
folder = 'projects/neat-planet-379122/assets/ten_chip_Colorado_II'
assets = ee.data.listAssets({'parent': folder})

#Get list of CCDC files from GEE
files = []
filestemp = assets['assets']
for n in filestemp:
    files.append(n['id'])

count = 1

#loop over CCDC Files to make ETa
for ccdfile in files:
    #Get Dates from files
    tstring = ccdfile
    year = str(tstring[-8:-4])
    month = str(tstring[-4:-2])
    day = str(tstring[-2:])
    
    #Get input data from CCDC 
    inimage = ee.Image(ccdfile)
    brightness = inimage.select('b1')
    ndvi = inimage.select('b2')
    ndwi = inimage.select('b3')
    water = inimage.select('b4')
    lst = CCDC_lst(brightness, ndvi)#getLST
    
    #running ssebop on ccdc files 
    input_img = ee.Image([ee.Image(lst), ee.Image(ndvi), ee.Image(ndwi), ee.Image(water)]) \
        .rename(['lst', 'ndvi', 'ndwi','qa_water']) \
        .set({
        #'system:index': 'LC08_044033_20170716',
            'system:time_start': ee.Date.fromYMD(int(year), int(month), int(day)).millis()}) #Fix Date
    
    # Build the SSEBop object from the CCDC files image
    model_obj = ssebop.Image(
        input_img, 
        tcorr_source='FANO',
        et_reference_source='projects/openet/reference_et/gridmet/daily',
        et_reference_band='etr',
        et_reference_factor=1.0,
        et_reference_resample='nearest',
        )


    #export LST to google Drive
    desc = 'ccdLST' + ccdfile[-15:]
    task_config = {
       # 'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_ccdLST_2019_II',
        'image': lst,
        'scale':30,
        'description': '5070_ccdc_ssebop_LST',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    #export ET Fraction to google Drive
    desc = 'ccdETFraction' + ccdfile[-15:]
    etF = model_obj.et_fraction
    task_config = {
       # 'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_ccdcETF_2019_II',
        'image': etF,
        'scale':30,
        'description': '5070_ccdc_ssebop_etFraction',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    
    #export ETA to google Drive
    desc = 'ccdETa' + ccdfile[-15:]
    eta = model_obj.et
    task_config = {
       # 'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_ccdcETA_2019_II',
        'image': eta,
        'scale':30,
        'description': '5070_ccdc_ssebop_ETa',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    print(count)
    count = count + 1



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
